In [1]:
import mmdet.apis
import mmcv
import torch
import pandas as pd
import cv2 as cv
import os
import numpy as np
from torch import nn
from pathlib import Path
from mmdet.apis.inference import _prepare_data
from mmdet.datasets.transforms import ImageTransform

In [2]:
def ls(self):
    return sorted(self.iterdir())

Path.ls = ls

In [3]:
class Detector:
    """
    built using: https://github.com/open-mmlab/mmdetection/tree/master/configs/htc
    (HTC + DCN + ResNeXt-101-FPN, mAP=50.7 model)
    
    Might take some fiddling to make it work with any mmdetection model
    
    config_file: "htc/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e.py"
    checkpoint_file: "htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c.pth"
    class_restrictions: list of classes to detect, others are discarded. if None, it will detect all classes
    
    """
    
    def __init__(self, config_path, model_path, verbose_interval=None, class_restrictions=set([2,3,5,7])):
        self.model = init_detector(config_path, model_path, device='cuda:0')
        self.class_labels = self.model.CLASSES
        self.class_restrictions = class_restrictions
        
        self.verbose_interval = verbose_interval
        
    def detect_objects(self, img):
        """
        Runs object detection on an image.
        Returns bounding boxes [x1, y1, x2, y2], class labels, and confidence scores
        """
        
        results, segment = inference_detector(self.model, img) # I think they are segmentations, not sure though

        bbox_and_scores = np.vstack(results)
        bboxes, scores = bbox_and_scores[:, :4], bbox_and_scores[:, 4]
        labels = np.concatenate([[i] * len(bbox) for i, bbox in enumerate(results)]).astype(int)
        
        order = np.argsort(scores)[::-1] # sort
        return bboxes[order], labels[order], scores[order]
    
    def detect_crop(self, img, crop_boxes):
        """
        Splits an image into boxes, upscales them, performs detection, downscales detections, merges detections
        
        img: numpy array of image
        crop_boxes: list of crop bounding boxes [x1, y1, x2, y2]
        
        returns: detection results [[x1, y1, x2, y2, score, class], ...]
        """
        pil_img = PIL.Image.fromarray(img)
        crops = crop_image(pil_img, crop_boxes)
        resized, biggest = resize_crops(crops)
        
        resized_np = (np.array(img) for img in resized)
        
        crop_results = self.detect_images(resized_np, verbose=False)
        
        bboxes = []
        for i, x1, y1, x2, y2, score, cls in crop_results.values:
            bboxes.append(cropped_detection_to_original((x1, y1, x2, y2), crop_boxes[int(i)], biggest))
        bboxes = np.array(bboxes)
        
        scores = crop_results["score"].values
        labels = crop_results["class"].values
        
        order = np.argsort(scores)[::-1] # sort
        return bboxes[order], labels[order], scores[order]

    
    
    def detect_images(self, images, frames=None, crop_boxes=None, verbose=True):
        """
        images: iterable of numpy array images
        frames: iterable of frame numbers/names corresponding to images
        crop_boxes: if provided, will crop and rescale for detection
        verbose: override for self.verbose_interval
        
        Returns dataframe with detection results
        """


        results = []
        for i, img in enumerate(images):
            frame = i if frames is None else frames[i]

            if crop_boxes is not None:
                bboxes, labels, scores = self.detect_crop(img, crop_boxes)
            else:
                bboxes, labels, scores = self.detect_objects(img)
            
            for (x1, y1, x2, y2), cls, score in zip(bboxes, labels, scores):
                if self.class_restrictions and cls not in self.class_restrictions:
                    continue 
                
                results.append([frame, x1, y1, x2, y2, score, cls])
                
            if verbose and self.verbose_interval and (i % self.verbose_interval) == 0:
                print(f"Detecting image: {frame}")
#                 print(*results[-5:], sep="\n")
                
        return pd.DataFrame(data=results, columns=["frame", "x1", "y1", "x2", "y2", "score", "class"])
                
    
    def label_image(self, img, bboxes, labels, scores, score_thresh=0.1):
        """
        Returns an image with bounding boxes drawn on it
        """
        
        
        def get_class_color(cls):
            return (0, 0, 255) # todo
        
        img = np.copy(img)
        
        for (x1, y1, x2, y2), cls, score in zip(bboxes, labels, scores):
            if score < score_thresh:
                continue
            
            col = get_class_color(cls)
            cv.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), col, 1) # draw boxes
            
        return img

In [72]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, images_folder, annotation_path, transforms=None):
        
        self.annotations = pd.read_csv(annotation_path)
        self.length = len(self.annotations["frame"].unique())
        
        self.transforms = transforms
        
        # temp
        self.flag = np.array([0])
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        img_path = os.path.join(images_folder, f"img{idx+1:0>5}.jpg")
        
        img = cv.cvtColor(cv.imread(img_path), cv.COLOR_BGR2RGB)
        
        boxes = self.annotations.loc[self.annotations.frame == idx + 1, ["x1", "y1", "x2", "y2"]].values
        
        if self.transforms:
            pass # todo
        
#         return {"img": img, "gt_bboxes": boxes, "img_meta": None}
#         return torch.Tensor(img).transpose(0, 2)

        data = _prepare_data(img, img_transform=tfm, cfg=cfg, device='cuda')
        data["img"] = data["img"][0][0]
        data["img_meta"] = data["img_meta"][0][0]
        data["gt_bboxes"] = torch.Tensor(boxes).to("cuda")
        data["gt_labels"] = torch.Tensor([0] * len(boxes)).to("cuda")
        
        return data

In [5]:
detector_config_path = "/data/modules/mmdetection/configs/htc/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e.py"
detector_model_path = "/data/modules/mmdetection/checkpoints/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c.pth"

images_folder = "/data/data/UE_DETRAC/train/images/MVI_20011"
annotations_path = "/data/data/UE_DETRAC/train/annotations/MVI_20011_v3.csv"

#### Config

In [76]:
cfg = mmcv.Config.fromfile(detector_config_path)
cfg.gpus = 1

In [100]:
model.train_cfg = cfg.train_cfg

In [30]:
tfm = ImageTransform(
    size_divisor=cfg.data.test.size_divisor,
    **cfg.img_norm_cfg
)

In [31]:
test_data = TrainDataset(images_folder, annotations_path)[0]

In [73]:
dataset = TrainDataset(images_folder, annotations_path)
dataset.length = 10

dl = torch.utils.data.DataLoader(dataset, batch_size=4)
test_batch = next(iter(dl))

In [74]:
test_batch["img"].shape

torch.Size([4, 3, 768, 1344])

In [33]:
model.bbox_head[0].loss_bbox()

TypeError: forward() missing 2 required positional arguments: 'pred' and 'target'

In [19]:
img_preped['img'][0].cpu().numpy()

array([[[[-0.8335474 , -0.76504844, -0.6965494 , ...,  0.        ,
           0.        ,  0.        ],
         [-0.8164227 , -0.7479237 , -0.7136742 , ...,  0.        ,
           0.        ,  0.        ],
         [-0.7992979 , -0.76504844, -0.76504844, ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],

        [[-0.9152661 , -0.8277311 , -0.7752101 , ...,  0.        ,
           0.        ,  0.        ],
         [-0.8977591 , -0.8277311 , -0.7927171 , ...,  0.        ,
           0.        ,  0.        ],
         [-0.8977591 , -0.8452381 , -0.8452381 , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        

+ Train MM Detector
    + To start with, lets pass some data to model and get result
        + ~~Had to figure out what format the model takes~~
        + ~~Fix error: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.~~
            + Solution was to use no_grad
    + Train model

In [94]:
test_path = Path(images_folder).ls()[0]
test_img = cv.imread(str(test_path))

with torch.no_grad():
    img_preped = _prepare_data(test_img, img_transform=tfm, cfg=cfg, device='cuda')
    model = mmdet.apis.init_detector(detector_config_path, detector_model_path)

/data/modules/mmdetection/mmdet/apis/inference.py:39: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [95]:
img_preped#["img"][0].shape

{'img': [tensor([[[[-0.8335, -0.7650, -0.6965,  ...,  0.0000,  0.0000,  0.0000],
            [-0.8164, -0.7479, -0.7137,  ...,  0.0000,  0.0000,  0.0000],
            [-0.7993, -0.7650, -0.7650,  ...,  0.0000,  0.0000,  0.0000],
            ...,
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
  
           [[-0.9153, -0.8277, -0.7752,  ...,  0.0000,  0.0000,  0.0000],
            [-0.8978, -0.8277, -0.7927,  ...,  0.0000,  0.0000,  0.0000],
            [-0.8978, -0.8452, -0.8452,  ...,  0.0000,  0.0000,  0.0000],
            ...,
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
  
           [[-0.7238, -0.6367, -0.5844,  ...,  0.0000,  0.0000,

In [85]:
test_batch["img_meta"]["pad_shape"]

[tensor([768, 768, 768, 768]),
 tensor([1344, 1344, 1344, 1344]),
 tensor([3, 3, 3, 3])]

In [101]:
with torch.no_grad():
#     test_batch = next(iter(dl)).cuda()
    res = model(img_preped["img"][0].cuda(), img_meta=None, gt_bboxes=[], gt_labels=None)
#     res = model(**test_batch)

TypeError: object of type 'NoneType' has no len()

In [58]:
bboxes, segments = res

Structure of bboxes

outer array: 80 classes
inner array: variable list of bbounding boxes. May be empty

Structure of segments
outer array: 80 classes
inner array: variable list (which corresponds to same element in bbox, with a base64 encoded segment map(?))

In [102]:
model.forward_train(**img_preped, )

ModuleList(
  (0): SharedFCBBoxHead(
    (loss_cls): CrossEntropyLoss()
    (loss_bbox): SmoothL1Loss()
    (fc_cls): Linear(in_features=1024, out_features=81, bias=True)
    (fc_reg): Linear(in_features=1024, out_features=4, bias=True)
    (shared_convs): ModuleList()
    (shared_fcs): ModuleList(
      (0): Linear(in_features=12544, out_features=1024, bias=True)
      (1): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (cls_convs): ModuleList()
    (cls_fcs): ModuleList()
    (reg_convs): ModuleList()
    (reg_fcs): ModuleList()
    (relu): ReLU(inplace)
  )
  (1): SharedFCBBoxHead(
    (loss_cls): CrossEntropyLoss()
    (loss_bbox): SmoothL1Loss()
    (fc_cls): Linear(in_features=1024, out_features=81, bias=True)
    (fc_reg): Linear(in_features=1024, out_features=4, bias=True)
    (shared_convs): ModuleList()
    (shared_fcs): ModuleList(
      (0): Linear(in_features=12544, out_features=1024, bias=True)
      (1): Linear(in_features=1024, out_features=1024, bias=

In [128]:
with torch.no_grad():
    x = model.extract_feat(img_preped["img"][0].cuda())
    rpn_outs = model.rpn_head(x)

In [129]:
rpn_outs + (img_preped[""])

([tensor([[[[ -3.7820,  -6.4972,  -8.4929,  ...,  -6.9590,  -4.4322,  -2.8508],
            [ -5.0382,  -8.4896, -12.1099,  ...,  -8.6000,  -5.7916,  -3.3913],
            [ -6.4024, -10.5768, -14.6528,  ...,  -9.3783,  -6.0515,  -3.8563],
            ...,
            [ -3.7974,  -7.5573, -10.4115,  ...,  -8.6536,  -6.0861,  -3.6137],
            [ -3.3151,  -6.0205,  -8.1901,  ...,  -7.6077,  -5.4724,  -2.6905],
            [ -1.9459,  -4.3093,  -6.1055,  ...,  -5.1048,  -3.8047,  -1.7798]],
  
           [[ -4.4332,  -6.8498,  -7.8419,  ...,  -6.2754,  -4.4916,  -2.6131],
            [ -6.3837,  -8.8273, -10.2021,  ...,  -7.3937,  -6.5373,  -2.6491],
            [ -8.8243, -11.8337, -12.4647,  ...,  -8.7234,  -7.2880,  -3.6343],
            ...,
            [ -4.7637,  -7.0737,  -8.0387,  ...,  -3.5045,  -3.7192,  -1.1579],
            [ -3.3425,  -3.8121,  -3.2851,  ...,  -1.2565,  -1.7141,   0.8503],
            [ -0.8636,  -1.8936,  -2.0231,  ...,  -0.7244,  -0.4236,   1.8360]],
 

In [130]:
img_preped.keys()

dict_keys(['img', 'img_meta'])

In [136]:
model.forward_train(img_preped["img"][0].cuda(), img_preped["img_meta"], torch.Tensor(test_data["gt_bboxes"]), gt_labels=None)

RuntimeError: CUDA out of memory. Tried to allocate 72.00 MiB (GPU 0; 10.92 GiB total capacity; 7.77 GiB already allocated; 22.50 MiB free; 75.17 MiB cached) (malloc at /pytorch/c10/cuda/CUDACachingAllocator.cpp:267)
frame #0: std::function<std::string ()>::operator()() const + 0x11 (0x7f506e729441 in /home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x2a (0x7f506e728d7a in /home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x15bc0 (0x7f506c7a9bc0 in /home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #3: <unknown function> + 0x16247 (0x7f506c7aa247 in /home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #4: at::native::empty_cuda(c10::ArrayRef<long>, c10::TensorOptions const&)

In [135]:
test_data

{'img': array([[[ 75,  64,  62],
         [ 83,  72,  70],
         [ 82,  71,  69],
         ...,
         [133, 115, 111],
         [132, 114, 110],
         [132, 114, 110]],
 
        [[ 77,  66,  64],
         [ 82,  71,  69],
         [ 79,  68,  66],
         ...,
         [133, 115, 111],
         [132, 114, 110],
         [132, 114, 110]],
 
        [[ 76,  65,  63],
         [ 77,  66,  64],
         [ 73,  62,  60],
         ...,
         [134, 116, 112],
         [133, 115, 111],
         [133, 115, 111]],
 
        ...,
 
        [[ 57,  48,  43],
         [ 57,  48,  43],
         [ 94,  85,  80],
         ...,
         [116, 113, 108],
         [119, 116, 111],
         [140, 137, 132]],
 
        [[ 61,  54,  48],
         [ 56,  49,  43],
         [ 89,  82,  76],
         ...,
         [124, 120, 117],
         [132, 128, 125],
         [148, 144, 141]],
 
        [[ 60,  53,  47],
         [ 53,  46,  40],
         [ 85,  78,  72],
         ...,
         [146, 142, 1

---
# Using coco format dataset

In [3]:
import mmdet
from mmdet.datasets.coco import CocoDataset

In [4]:
ue_dataset_path = "/data/data/UE_DETRAC/train/annotations/MVI_20011.json"
ue_images_path = ""

In [ ]:
CocoDataset(ue_dataset_path, images_folder, (1333, 800), {}, with_mask=False, with_crowd=False, )

In [6]:
import sys
print(*sys.path, sep="\n")

/home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python36.zip
/home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6
/home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/lib-dynload
/opt/conda/lib/python3.6

/home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/site-packages
/data/modules/mmdetection
/home/adam/.local/share/virtualenvs/data-I7nS9QO2/lib/python3.6/site-packages/IPython/extensions
/home/adam/.ipython


In [15]:
%pip


Usage:   
  /home/adam/.local/share/virtualenvs/data-I7nS9QO2/bin/python -m pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper command used for command completion.
  help                        Show help for commands.

General Options:
  -h, --help                  Show help.
